In [1]:
!pip install tensorflow==2.15.0 tensorflow-hub keras==2.15.0
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import pandas as pd

In [2]:
train_data, validation_data, test_data = tfds.load(name="ag_news_subset",
                                                   split=('train[:60%]', 'train[60%:]', 'test'),
                                                   as_supervised=True)

In [3]:
train_data
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

GPU is NOT AVAILABLE


In [4]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [5]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.',
       b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.',
       b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.',
       b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.',
       b'London, England (Sports Network) - England midfielder Steven Gerrard injured his groin late in Thursday #39;s training session, but is hopeful he will be ready for Saturday #39;s World Cup qualifier against Austri

In [6]:
 train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([3, 1, 2, 3, 1, 0, 3, 0, 0, 1])>

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

In [8]:
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)

In [9]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.4586596 , -0.22270747,  1.7708323 ,  1.9534097 ,  1.2787155 ,
         0.10514727,  1.6557657 , -1.69676   , -2.3956175 , -3.1235514 ,
         1.9141243 ,  1.7303494 , -2.2757962 ,  0.55892617,  0.9924717 ,
        -1.8745328 ,  1.7581484 ,  0.36097184,  2.1421185 , -1.9464955 ],
       [-3.1687405 , -0.8680503 ,  0.22489524,  0.8019142 ,  1.4973278 ,
         1.4691905 ,  0.34958157, -1.1848819 , -0.35689202,  1.2081021 ,
        -1.6422368 , -1.648375  ,  0.58841795, -0.27546963,  0.7328136 ,
         1.7875123 , -0.27011397, -1.1866195 , -1.0566308 ,  2.1479883 ],
       [-1.2233101 , -0.37048745,  1.0669774 ,  0.8257882 ,  1.7866129 ,
        -2.4578087 ,  0.9420228 , -0.17693122, -0.44748542,  0.2637198 ,
         1.5832659 , -0.99717855, -1.3851663 ,  0.28552565,  0.604694  ,
         0.7792836 ,  2.8988218 , -2.5159223 ,  0.42583808,  0.54979295]],
      dtype=float32)>

In [10]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
history = model.fit(train_data.shuffle(10000).batch(500),
                    epochs=25,
                    validation_data=validation_data.batch(500),
                    verbose=1)

Epoch 1/25
144/144 [==============================] - 11s 74ms/step - loss: -83819.0312 - accuracy: 0.2498 - val_loss: -87938.1562 - val_accuracy: 0.2522
Epoch 2/25
144/144 [==============================] - 4s 28ms/step - loss: -93678.0703 - accuracy: 0.2498 - val_loss: -98039.0703 - val_accuracy: 0.2522
Epoch 3/25
144/144 [==============================] - 4s 28ms/step - loss: -104162.5859 - accuracy: 0.2498 - val_loss: -108752.3203 - val_accuracy: 0.2522
Epoch 4/25
144/144 [==============================] - 6s 38ms/step - loss: -115278.4531 - accuracy: 0.2497 - val_loss: -120099.0000 - val_accuracy: 0.2522
Epoch 5/25
144/144 [==============================] - 6s 37ms/step - loss: -127040.9219 - accuracy: 0.2497 - val_loss: -132091.9531 - val_accuracy: 0.2522
Epoch 6/25
144/144 [==============================] - 4s 28ms/step - loss: -139458.3281 - accuracy: 0.2497 - val_loss: -144731.7969 - val_accuracy: 0.2522
Epoch 7/25
144/144 [==============================] - 6s 37ms/step - loss

In [17]:
results = model.evaluate(test_data.batch(500), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

16/16 - 0s - loss: -5.2408e+05 - accuracy: 0.2507 - 246ms/epoch - 15ms/step
loss: -524082.125
accuracy: 0.251
